In [67]:
import re
import spacy
import nltk
from itertools import permutations
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score

In [69]:
#Loads a list of words from a file.
def load_words(filepath):
    try:
        with open(filepath, 'r') as f:
            words = [word.strip().lower() for word in f]
        return words
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
        return []

In [71]:
#Loads a corpus of text from a file.
def load_corpus(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            corpus = f.read()
        return corpus
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
        return ""

In [95]:
# Load variables from respective text files.
nlp = spacy.load("en_core_web_sm")
word_list = load_words("words.txt")
corpus = load_corpus("corpus.txt")

In [9]:
#Counts bigram frequencies in a corpus and loads it into a dictionary[bigram:frequency].
def count_bigrams(corpus):
    corpus = re.sub(r'[^\w\s]', ' ', corpus).lower()
    words = corpus.split()
    bigram_counts = defaultdict(int)
    for i in range(len(words) - 1):
        bigram = (words[i], words[i + 1])
        bigram_counts[bigram] += 1
    return bigram_counts

In [11]:
#Calculates total number of bigrams and bigram probabilities of each bigram and loads it into a dictionary[bigram:probability].
def calculate_bigram_probabilities(bigram_counts):
    total_bigrams = sum(bigram_counts.values())
    bigram_probabilities = {}
    for bigram, count in bigram_counts.items():
        bigram_probabilities[bigram] = count / total_bigrams
    return bigram_probabilities

In [13]:
#Calculates the Levenshtein distance between two strings.
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

In [15]:
#Checks if a word is in the dictionary and suggests corrections using levenshtein distance.
def spell_check(word, word_list, max_distance=2):
    word = word.lower()
    if word in word_list:
        return word, []
    suggestions = []
    for w in word_list:
        distance = levenshtein_distance(word, w)
        if distance <= max_distance:
            suggestions.append((w, distance))
    suggestions.sort(key=lambda x: x[1])
    return word, [s[0] for s in suggestions]

In [17]:
#Contextual spell check using bigram probabilities.
def contextual_spell_check(sentence, word_list, bigram_probabilities, max_distance=2):
    doc = nlp(sentence)
    words = [token.text for token in doc]
    corrected_words = []

    for i, word in enumerate(words):
        if word.lower() in word_list:
            corrected_words.append(word)
            continue

        _, suggestions = spell_check(word, word_list, max_distance)

        if not suggestions:
            corrected_words.append(word)
            continue

        best_suggestion = suggestions[0]

        if i > 0:
            best_prob = 0
            for suggestion in suggestions:
                bigram = (words[i - 1].lower(), suggestion.lower())
                prob = bigram_probabilities.get(bigram, 0.000000000000001)
                if prob > best_prob:
                    best_prob = prob
                    best_suggestion = suggestion

        corrected_words.append(best_suggestion)

    return " ".join(corrected_words)

In [93]:
#Grammar productions.
grammar_str = f"""
    S -> NP VP | S Conj S | S PP
    NP -> Det N | Pronoun
    VP -> V | V NP | Be Adj
    PP -> P NP
    Conj -> 'and'
    P -> 'with'
    Det -> 'a' | 'his' | 'my'
    N -> 'violin' | 'dog' | 'microscope' | 'garden' | 'tail'
    V -> 'have' | 'has' | 'wags' | 'plays' | 'visits'
    Adj -> 'cute' | 'happy'
    Pronoun -> 'I' | 'He' | 'he' | 'They'
    Be -> 'is' | 'am'
"""

In [77]:
#Checks and corrects grammar by rearranging words using the production rules.
def check_and_correct_grammar(sentence, grammar_str):
    try:
        grammar = nltk.CFG.fromstring(grammar_str)
        parser = nltk.ChartParser(grammar)
        tokens = sentence.split()

        for tree in parser.parse(tokens):
            print("Grammatically correct:")
            tree.pretty_print()
            return True, tree, sentence

        print("Grammatically incorrect.")

        corrected_sentence = rearrange_words(tokens, grammar)
        if corrected_sentence:
            print(f"Corrected sentence: {corrected_sentence}")
            tokens1 = corrected_sentence.split()
            for tree in parser.parse(tokens1):
                tree.pretty_print()
            return False, None, corrected_sentence
        else:
            print("Unable to correct sentence by rearranging.")
            return False, None, sentence

    except ValueError as e:
        print(f"Error: {e}")
        return False, None, sentence
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False, None, sentence

In [79]:
#Attempts to rearrange words to match the grammar.
def rearrange_words(tokens, grammar):

    for perm in permutations(tokens):
        try:
            parser = nltk.ChartParser(grammar)
            for tree in parser.parse(perm):
                return " ".join(perm)
        except ValueError:
            pass
    return None

In [81]:
#Calculate bigram frequency and probability.
if not word_list or not corpus:
    exit()
bigram_counts = count_bigrams(corpus)
bigram_probabilities = calculate_bigram_probabilities(bigram_counts)

In [83]:
#Correct the inputted paragraph.
def correct_paragraph(paragraph, word_list, bigram_probabilities, contextual_spell_check):
    """Corrects a paragraph by processing each sentence."""
    sentences = nltk.sent_tokenize(paragraph)
    corrected_sentences = []

    for sentence in sentences:
        punctuation = []
        sentence_no_punct = sentence
        for match in re.finditer(r'[^\w\s]', sentence):
            punctuation.append((match.group(0), match.start()))
            sentence_no_punct = sentence_no_punct.replace(match.group(0), "")
            sentence = sentence_no_punct
        partially_corrected_sentence = contextual_spell_check(sentence, word_list, bigram_probabilities)
        print(f"\nChecking sentence: '{partially_corrected_sentence}'")
        is_correct, tree, corrected_sentence = check_and_correct_grammar(partially_corrected_sentence, grammar_str)
        offset = 1
        for char, pos in punctuation:
            corrected_sentence = corrected_sentence[:pos + offset] + char + corrected_sentence[pos + offset:]
            offset += 1
        corrected_sentences.append(corrected_sentence)

    corrected_paragraph = " ".join(corrected_sentences)
    return corrected_paragraph

In [89]:
#Get input.
paragraph = input("Enter a paragraph: ")
corrected_paragraph = correct_paragraph(paragraph, word_list, bigram_probabilities, contextual_spell_check)
print(f"Corrected paragraph: {corrected_paragraph}")
#Sample Input: I a have dog. He his tael wags. He is cuute and he plays.

Enter a paragraph:  I a have dog. He his tael wags. He is cuute and he plays.



Checking sentence: 'I a have dog'
Grammatically incorrect.
Corrected sentence: I have a dog
              S             
    __________|___           
   |              VP        
   |      ________|___       
   NP    |            NP    
   |     |         ___|___   
Pronoun  V       Det      N 
   |     |        |       |  
   I    have      a      dog


Checking sentence: 'He his tail wags'
Grammatically incorrect.
Corrected sentence: He wags his tail
              S              
    __________|___            
   |              VP         
   |      ________|___        
   NP    |            NP     
   |     |         ___|___    
Pronoun  V       Det      N  
   |     |        |       |   
   He   wags     his     tail


Checking sentence: 'He is cute and he plays'
Grammatically correct:
                 S                         
          _______|_________________         
         S            |            S       
    _____|___         |       _____|____    
   NP        VP   

In [97]:
#Evaluate using precision, recall and f1-score
def evaluate_predictions(corrected_text, ground_truth):
    corrected_tokens = corrected_text.split()
    ground_truth_tokens = ground_truth.split()

    min_len = min(len(corrected_tokens), len(ground_truth_tokens))
    corrected_tokens = corrected_tokens[:min_len]
    ground_truth_tokens = ground_truth_tokens[:min_len]

    y_true = [1 if ct == gt else 0 for ct, gt in zip(corrected_tokens, ground_truth_tokens)]
    y_pred = [1] * len(y_true)

    precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)
    return {"Precision": precision, "Recall": recall, "F1-score": f1}

with open("test.txt", "r") as file:
    test_corpus = file.read()
with open("ground.txt", "r") as file:
    ground_truth = file.read()

corrected_corpus = correct_paragraph(test_corpus, word_list, bigram_probabilities, contextual_spell_check)
metrics = evaluate_predictions(corrected_corpus, ground_truth)
print(metrics)


Checking sentence: 'They have a microscope'
Grammatically correct:
              S                    
    __________|___                  
   |              VP               
   |      ________|___              
   NP    |            NP           
   |     |         ___|______       
Pronoun  V       Det         N     
   |     |        |          |      
  They  have      a      microscope


Checking sentence: 'He his garden visits'
Grammatically incorrect.
Corrected sentence: He visits his garden
                S                
    ____________|___              
   |                VP           
   |       _________|___          
   NP     |             NP       
   |      |          ___|____     
Pronoun   V        Det       N   
   |      |         |        |    
   He   visits     his     garden


Checking sentence: 'He is cute and he has a violin'
Grammatically correct:
                      S                                 
          ____________|________________           